<a href="https://colab.research.google.com/github/Kim-JungHyun01/AIStudy24/blob/master/%ED%85%8C%EC%8A%A4%ED%8A%B8/%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 히스토그램 기반 그레이디언트 부스팅 회귀버전 설치
# !pip install xgboost
# !pip install pycaret
# 설치완료 -> 런타임재시작 -> 이설치문은 잠그기

# 데이터

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
print(data, "\n")
print(target)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]] 

[0. 0. 0. ... 1. 1. 1.]


In [3]:
# 훈련세트 80%, 테스트세트 20%
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

print("train : ", train_input.shape, " test : ", test_input.shape)

train :  (5197, 3)  test :  (1300, 3)


# 앙상블(RandomForestClassifier)
- 여러 단순한 모델을 **결합**하여 정확한 모델을 만드는 방법
- 데이터종류
    - 정형데이터 : 지금까지 학습한 수치자료가 있는 값
    - 비정형데이터 : 데이터베이스나 엑셀로 표현하기 어려운 데이터(텍스트데이터, 디카사진, mp3등) => 신경망 알고리즘

##랜덤 포레스트
- 결정 트리를 랜덤하게 만들어 결정트리(나무)숲을 만듬 => **최종 예측**
- 부트스트랩 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링
    - 부트스트랩 샘플 : 샘플이 들어있는 가방에서 샘플묶음을 뽑을 때, 뽑았던 샘플을 다시 가방에 넣고 이를 반복(중복허용)

- 분류모델(RandomForestClassifiter) : 기본적으로 전체 특성 개수의 제곱근만큼 특성을 선택
    - 즉, 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용
- 회귀모델(RandomForestRegressor) : 전체특성 사용

- 훈련
    - 분류 : 각 트리의 클래스별 확률을 평균하여 가장 높은 확율을 가진 클래스를 예측
        - 샘플을 몇개의 클래스 중 하나로 분류하는 문제
    - 회귀 : 단순히 각 트리의 예측을 평균
        - 임의의 어떤 숫자를 예측하는 문제
        
<img src = "https://velog.velcdn.com/images/chanhee-kang/post/e5b7b5fb-6da8-4d71-947c-62af9652be8c/image.png">

In [4]:
from sklearn.model_selection import cross_validate  #크로스검증용
from sklearn.ensemble import RandomForestClassifier #100개의 결정 트리 사용

rf = RandomForestClassifier(n_jobs=-1, random_state=42) # n_jobs = -1 모든 cpu사용

# return_train_score : 검증 점수와 훈련세트 점수의 반환여부
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 검증(0.997) => 100%에 가까움 => 과대적합

train :  0.9973541965122431  test :  0.8905151032797809


- 결정트리의 앙상블이기 때문에 결정트리가 제공하는 중요한 매개변수를 모두 제공함
- 장점: 특성 중요도를 계산(랜덤포레스트의 특성 중요도는 각 결정트리의 특성 중요소를 취합한 것)

In [18]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
"""
  'alcohol'   'sugar'     'pH'
[0.12345626 0.86862934 0.0079144 ] : 로지스틱회귀(이전)
[0.23167441 0.50039841 0.26792718] : 랜덤포레스트(현재)

중요도 : 알코올, pH 증가, 당도 감소
"""

[0.23167441 0.50039841 0.26792718]


달라진 이유
- 랜덤포레스트가 특성 일부를 랜덤하게 선택하여 결정트리를 훈련(중복허용)
- 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성잉 훈련에 기여할 기회가 됨(= 과대적합 줄임)

In [6]:
# RandomForestClassifiter 기능 중에 자체적으로 모델을 평가하는 점수를 얻을  수 있음
# OOB(Out of Bag) : 부트스트랩에 포함되지 않고 남은 샘플 -> 결정 트리 평가용(검증세트로 활용)
# oob_score : 랜덤포레스트는 각 결정트리의 OOB 점수를 평균하여 출력(oob_score_)
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state = 42)

rf.fit(train_input, train_target)
print(rf.oob_score_)    # 교차검증와 비슷한 결과를 얻음
# OOB점수를 사용하면 교차 검증을 대신할 수 있어 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있음

0.8934000384837406


## 엑스트라 트리(ExtraTreesClassifier)
- 부트스트랩 샘플을 사용하지 않고, 전체 훈련세트 이용
- 대신 노드를 분할할 때, 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함
- 결정트리 : splitter = 'random'
- 하나의 결정트리에서 특성을 무작위로 분할한다면 성능이 낮아지지만, 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증세트 점수 높이는 효과가 있음

In [17]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
"""
랜덤 : train :  0.9973541965122431  test :  0.8905151032797809
엑스 : train :  0.9974503966084433  test :  0.8887848893166506
결과 비슷하지만 처리속도가 ExtraTreesClassifier이 빠름
"""

train :  0.9974503966084433  test :  0.8887848893166506


In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅(Gradient Boosting)

- 기울기
- 깊이가 앝은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식의 앙상블
- sklearn의 GradientBoostingClassifiter는 기본적으로 깊이가 3인 결정 트리 100개 이용
- 과대적합에 강하고 일반적인 높은 일반화 성능을 기대
- Gradient : 경사 하강법을 사용하여 트리를 앙상블에 추가
    - 경사하강법 손실 함수를 산으로 정의하고 가장 낮은 곳으로 찾아 내려오는 과정
<br/><br/>
- 손실 함수
    - 분류 : 로지스틱 손실 함수
    - 회귀 : 평균제곱오차 함수
<br/><br/>
- 가장 낮은 곳으로 내려오는 방법은 모델의 가중치와 절편을 조금씩 바꾸는 것
- 결정트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동, 손실함수의 낮은 곳으로 천천히 이동
- n_jobs등 몇몇 매개변수가 없음(구형모델이라)
- 순서대로 트리를 추가하기 때문에 훈련속도가 좀 느림

<img src = "https://i.ytimg.com/vi/d6nRgztYWQM/maxresdefault.jpg" width = 500px>

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 train(0.888), test(0.872) => 과대적합 해결

train :  0.8881086892152563  test :  0.8720430147331015


In [10]:
from sklearn.ensemble import GradientBoostingClassifier
# n_estimators : 결정트리갯수설정(기본값 : 100)
# learning_rate : 학습률 (기본값 : 0.1)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 train(0.946) > test(0.878) => 검증차이 10%이상이내 => 가능(과대적합x)

train :  0.9464595437171814  test :  0.8780082549788999


In [11]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 그레이디언트 부스팅은 랜덤포레스트보다 조금 더 높은 성능
# 하지만 순서대로 트리를 추가하기 때문에 훈련속도가 좀 느림
# GradientBoostingClassifier에는 n_jobs 매개변수 없음(구형모델)

[0.15887763 0.6799705  0.16115187]


### 히스토그램기반 그레이디언트 부스팅(Histogram Gradient)
- from sklearn.ensemble import **Hist**GradientBoostingClassifier
- 그레이디언트 부스팅의 개선버전
- 입력 특성을 256(2^8)구간으로 나눔
    
    -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 특히 256구간 중에서 하나를 뗴어 놓고 **누락된 값**을 위해서 사용함
- 기본 매개변수에서 안정적인 성능을 얻을 수 있음
- 트리의 개수지정가능
- n_estimators 대신 max_iter를 사용(성능항상용)

#### 분류버전(HistGradientBoostingClassifier)

In [12]:
# from sklearn.experimental import enable_hist_gradient_boosting #아직 개발중
from sklearn.ensemble import HistGradientBoostingClassifier #분류용

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 train(0.932) test(0.880) => 과대적합 억제

train :  0.9321723946453317  test :  0.8801241948619236


#### 특성중요도 확인(permutation_importance)
- 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화되는지 관찰
- n_repeats = ??? : 랜덤하게 섞을 수 있는 횟수(기본값 : 5)

In [13]:
from sklearn.inspection import permutation_importance

# 히스토그램 기반의 훈련진행
hgb.fit(train_input, train_target)

# n_repeats = 10 : 랜덤하게 섞을 수 있는 횟수 (기본값 : 5)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [14]:
#test
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


#### 회귀버전(XGBClassifier)
- sklearn 말도고 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 다수 존재
- XGBoost 대표적임 -> 크로스검증이 가능(cross_validate())

In [20]:
from xgboost import XGBClassifier

# tree_method='hist' : 히스토그램 기반 그레이디언트 부스팅용
xgb = XGBClassifier(tree_method = 'hist', random_state  = 42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score = True)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 train(0.956) test(0.878) => 과대적합 억제

train :  0.9558403027491312  test :  0.8782000074035686


In [23]:
from sklearn.inspection import permutation_importance

# 히스토그램 기반의 훈련진행
xgb.fit(train_input, train_target)

# n_repeats = 10 : 랜덤하게 섞을 수 있는 횟수 (기본값 : 5)
result = permutation_importance(xgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.11264191 0.25932269 0.10800462]


In [21]:
# LGBMClassifiter ms에서 만든 LightGBM
from lightgbm import LGBMClassifier #분류

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("train : ", np.mean(scores['train_score']), " test : ", np.mean(scores['test_score']))
# 결론 train(0.936) test(0.880)

train :  0.935828414851749  test :  0.8801251203079884


## 결론

In [ ]:
#각 모델별 특성 중요도 확인
"""
  'alcohol'   'sugar'     'pH'
[0.12345626 0.86862934 0.0079144 ] : 로지스틱회귀
[0.23167441 0.50039841 0.26792718] : 랜덤포레스트       train :  0.997  test :  0.891
[0.20183568 0.52242907 0.27573525] : 엑스트라트리       train :  0.997  test :  0.889
[0.15887763 0.6799705  0.16115187] : 그레이디언트       train :  0.946  test :  0.878
[0.08876275 0.23438522 0.08027708] : 히스토그램(분류)   train :  0.932  test :  0.880
[0.11264191 0.25932269 0.10800462] : 히스토그램(회귀)   train :  0.956  test :  0.878
[0.05969231 0.20238462 0.049     ] : test
"""